In [33]:
import tomobase
import numpy as np
from tomobase.data import Volume
from tomobase import phantoms
import glob
import tifffile

arr = tifffile.imread(r'D:\Assets\nottwisted_filtered.tif')
volume = Volume(arr)
# normalize between 0 and 1
volume.data = (volume.data - np.min(volume.data)) / (np.max(volume.data) - np.min(volume.data))
v1 = volume.to_file(r'D:\Assets\nottwisted.rec')

#v1 = Volume.from_file('sinogram_1.h5')
#v2 = Volume.from_file('sinogram_2.h5')
#v3 = phantoms.nanocage()






In [37]:
import stackview

v1 = Volume.from_file(r'D:\Assets\twisted.rec')
v2 = Volume.from_file(r'D:\Assets\nottwisted.rec')
v3 = phantoms.nanocage()

filenames = [r'D:\Assets\twisted.rec', r'D:\Assets\nottwisted.rec', r'D:\Assets\nanocage.rec']
i=0
for volume in [v1, v2, v3]:
    # pad array to size of longest dimension
    max_dim = max(volume.data.shape)
    padded_shape = tuple(max_dim if dim < max_dim else dim for dim in volume.data.shape)
    padded_volume = np.zeros(padded_shape, dtype=volume.data.dtype)
    #centre sample in padded array
    slices = tuple(slice((max_dim - dim) // 2, (max_dim - dim) // 2 + dim) for dim in volume.data.shape)
    padded_volume[slices] = volume.data
    padded_volume = Volume(padded_volume, metadata=volume.metadata)
    
    #normalize between 0 and 1
    padded_volume.data = (padded_volume.data - np.min(padded_volume.data)) / (np.max(padded_volume.data) - np.min(padded_volume.data))
    padded_volume.to_file(filenames[i])
    i += 1

In [55]:
from tomobase.tiltschemes import GRS, Binary, Incremental

v1 = Volume.from_file(r'D:\Assets\twisted.rec')
v2 = Volume.from_file(r'D:\Assets\nottwisted.rec')
v3 = phantoms.nanocage()

ts_list = []
for volume in [v1,v2]:
    volume.data = (volume.data - np.min(volume.data)) / (np.max(volume.data) - np.min(volume.data))
    for i in range(3):
        match i:
            case 0:
                ts = Incremental(-64,64,2)
            case 1:
                ts = Binary(-64, 64, 8)
            case 2:
                ts = GRS(-64, 64, 0)
        angles = np.array([ts.get_angle() for i in range(65)])
        tiltseries = tomobase.processes.project(volume, angles)
        ts_list.append(tiltseries)


100%|██████████| 311/311 [00:00<00:00, 343.04it/s]


In [56]:
import copy
from skimage.metrics import structural_similarity as ssim
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

df = pd.DataFrame(columns=['SSIM', 'RMSE'])

vol_list = [v1, v2]
ts_per_vol = 3  # You have 3 tiltseries per volume

# Group ts_list into sublists for each volume
ts_groups = [ts_list[i*ts_per_vol:(i+1)*ts_per_vol] for i in range(len(vol_list))]

for volume, ts_group in zip(vol_list, ts_groups):
    for ts in ts_group:
        for i in range(3):
            match i:
                case 0:
                    ts_copy = copy.deepcopy(ts)
                    rec = tomobase.processes.reconstruct.optomo_reconstruct(ts_copy, iterations=100, weighted=False)
                case 1:
                    ts_copy = copy.deepcopy(ts)
                    rec = tomobase.processes.reconstruct.optomo_reconstruct(ts_copy, iterations=100, weighted=True)
                case 2:
                    ts_copy = copy.deepcopy(ts)
                    for j in range(len(ts.angles)):
                        if j != 0 and ts_copy.angles[j] < ts_copy.angles[j-1]:
                            ts_copy.angles[j] -= 1.0
                    rec = tomobase.processes.reconstruct.optomo_reconstruct(ts_copy, iterations=100, weighted=True)
            
            ssim_val = ssim(volume.data, rec.data, data_range=volume.data.max() - volume.data.min())
            rmse_val = np.sqrt(np.mean((volume.data - rec.data) ** 2))
            df1 = pd.DataFrame({'SSIM': [ssim_val], 'RMSE': [rmse_val]})
            df = pd.concat([df, df1], ignore_index=True)
            print(i)
            print(df)
    print(df)

100%|██████████| 257/257 [02:29<00:00,  1.72it/s]
2025-07-01 14:53:10,469 - INFO - type of volume: <class 'tomobase.data.volume.Volume'>
C:\Users\TCraig\AppData\Local\Temp\ipykernel_29984\3146290649.py:36: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, df1], ignore_index=True)


0
       SSIM      RMSE
0  0.923929  0.043945


100%|██████████| 257/257 [02:31<00:00,  1.69it/s]
2025-07-01 14:55:45,190 - INFO - type of volume: <class 'tomobase.data.volume.Volume'>


1
       SSIM      RMSE
0  0.923929  0.043945
1  0.935602  0.039491


100%|██████████| 257/257 [02:33<00:00,  1.67it/s]
2025-07-01 14:58:21,607 - INFO - type of volume: <class 'tomobase.data.volume.Volume'>


...

0
        SSIM      RMSE
0   0.923929  0.043945
1   0.935602  0.039491
2   0.935602  0.039491
3   0.921723  0.044847
4   0.933644  0.040188
5   0.933211  0.040279
6   0.922621  0.044459
7   0.934470  0.039898
8   0.934492  0.039887
9   0.940816  0.030367
10  0.950046  0.028363
11  0.950046  0.028363
12  0.938491  0.031081
13  0.947974  0.028841
14  0.944450  0.029903
15  0.938500  0.030953


100%|██████████| 311/311 [03:44<00:00,  1.39it/s]
2025-07-01 15:45:25,454 - INFO - type of volume: <class 'tomobase.data.volume.Volume'>


1
        SSIM      RMSE
0   0.923929  0.043945
1   0.935602  0.039491
2   0.935602  0.039491
3   0.921723  0.044847
4   0.933644  0.040188
5   0.933211  0.040279
6   0.922621  0.044459
7   0.934470  0.039898
8   0.934492  0.039887
9   0.940816  0.030367
10  0.950046  0.028363
11  0.950046  0.028363
12  0.938491  0.031081
13  0.947974  0.028841
14  0.944450  0.029903
15  0.938500  0.030953
16  0.948671  0.028672


100%|██████████| 311/311 [03:44<00:00,  1.38it/s]
2025-07-01 15:49:14,848 - INFO - type of volume: <class 'tomobase.data.volume.Volume'>


2
        SSIM      RMSE
0   0.923929  0.043945
1   0.935602  0.039491
2   0.935602  0.039491
3   0.921723  0.044847
4   0.933644  0.040188
5   0.933211  0.040279
6   0.922621  0.044459
7   0.934470  0.039898
8   0.934492  0.039887
9   0.940816  0.030367
10  0.950046  0.028363
11  0.950046  0.028363
12  0.938491  0.031081
13  0.947974  0.028841
14  0.944450  0.029903
15  0.938500  0.030953
16  0.948671  0.028672
17  0.946448  0.030037
        SSIM      RMSE
0   0.923929  0.043945
1   0.935602  0.039491
2   0.935602  0.039491
3   0.921723  0.044847
4   0.933644  0.040188
5   0.933211  0.040279
6   0.922621  0.044459
7   0.934470  0.039898
8   0.934492  0.039887
9   0.940816  0.030367
10  0.950046  0.028363
11  0.950046  0.028363
12  0.938491  0.031081
13  0.947974  0.028841
14  0.944450  0.029903
15  0.938500  0.030953
16  0.948671  0.028672
17  0.946448  0.030037
